In [78]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_colwidth', None)

In [9]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 1: Scrape listing links
i = 4235
url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
houselinks = []
while True:  # Adjust the range as needed
    i += 1
    r = requests.get(url, headers=headers)

    url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
    print(f'Page {i} status code:', r.status_code)
    soup = BeautifulSoup(r.content, 'lxml')

    # Check if no listings found
    h3_element = soup.find('h3', class_='css-1nw1os0 e1ws6l2x3')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found. Stopping the scraping process.")
        break

    listing_container = soup.find_all('section', class_='eeungyz1 css-hqx1d9 e12fn6ie0')
    if not listing_container:
        print(f"No listing container found on page {i}")
    else:
        print(f"Found {len(listing_container)} listing containers on page {i}")

    # Extract individual listing URLs
    for section in listing_container:
        links = section.find_all('a', class_='css-16vl3c1 e17g0c820')
        for link in links:
            href = link['href']
            if href and href not in houselinks:
                # Ensure the URL is complete
                if not href.startswith('http'):
                    href = 'https://www.otodom.pl' + href
                houselinks.append(href)

print(f"Total houselinks found: {len(houselinks)}")

# Step 2: Scrape details from each listing
houselist = []
for link in houselinks:
    r = requests.get(link, headers=headers)
    if r.status_code == 403:
        print(f"Access denied for link: {link}")
        continue

    soup = BeautifulSoup(r.text, 'lxml')
    body = soup.select_one('main')
    if not body:
        print(f"No main section found for {link}")
        continue

    # Check if no listings found
    h3_element = body.find('h3', class_='css-1nw1os0 e1ysrxc23')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found on this detail page. Stopping the scraping process.")
        break

    try:
        NumerId = link

        nazwa = body.find('h1', class_='css-1wnihf5 e1j8g12x8')
        nazwa = nazwa.text.strip() if nazwa else "N/A"

        cenaM2 = body.find('div', class_='css-1h1l5lm e1j8g12x9')
        cenaM2 = cenaM2.text.strip() if cenaM2 else "N/A"

        cena = body.find('strong', class_='css-t3wmkv e9aa0kv0')
        cena = cena.text.strip() if cena else "N/A"

        adres = body.find('a', class_='eozeyij0 css-1helwne e1p0dzoz0')
        adres = adres.text.strip() if adres else "N/A"

        powierzchnia = body.find('div', {'aria-label': 'Powierzchnia', 'class': 'css-1ivc1bc e26jmad1'})
        powierzchnia = powierzchnia.text.strip() if powierzchnia else "N/A"

        formaWlasnosci = body.find('div', {'aria-label': 'Forma własności', 'class': 'css-1ivc1bc e26jmad1'})
        formaWlasnosci = formaWlasnosci.text.strip() if formaWlasnosci else "N/A"

        LiczbaPokoi = body.find('div', {'aria-label': 'Liczba pokoi', 'class': 'css-1ivc1bc e26jmad1'})
        LiczbaPokoi = LiczbaPokoi.text.strip() if LiczbaPokoi else "N/A"

        stanWykonczenia = body.find('div', {'aria-label': 'Stan wykończenia','class':'css-1ivc1bc e26jmad1'})
        stanWykonczenia = stanWykonczenia.text.strip() if stanWykonczenia else "N/A"

        pietro = body.find('div', {'aria-label': 'Piętro', 'class': 'css-1ivc1bc e26jmad1'})
        pietro = pietro.text.strip() if pietro else "N/A"

        BalkonOgrodTaras = body.find('div', {'aria-label': 'Balkon / ogród / taras', 'class': 'css-1ivc1bc e26jmad1'})
        BalkonOgrodTaras = BalkonOgrodTaras.text.strip() if BalkonOgrodTaras else "N/A"

        Czynsz = body.find('div', {'aria-label': 'Czynsz', 'class': 'css-1ivc1bc e26jmad1'})
        Czynsz = Czynsz.text.strip() if Czynsz else "N/A"
        
        MiejsceParkingowe = body.find('div', {'aria-label': 'Miejsce parkingowe', 'class': 'css-1ivc1bc e26jmad1'})
        MiejsceParkingowe = MiejsceParkingowe.text.strip() if MiejsceParkingowe else "N/A"
        
        ObslugaZdalna = body.find('div', {'aria-label': 'Obsługa zdalna', 'class': 'css-1ivc1bc e26jmad1'})
        ObslugaZdalna = ObslugaZdalna.text.strip() if ObslugaZdalna else "N/A"

        ogrzewanie = body.find('div', {'aria-label': 'Ogrzewanie', 'class': 'css-1wi2w6s e26jmad5'})
        ogrzewanie = ogrzewanie.text.strip() if ogrzewanie else "N/A"

        Rynek = body.find('div', {'aria-label': 'Rynek', 'class': 'css-tpkder e26jmad1'})
        Rynek = Rynek.text.strip() if Rynek else "N/A"

        TypOgloszeniodawcy = body.find('div', {'aria-label': 'Typ ogłoszeniodawcy', 'class': 'css-tpkder e26jmad1'})
        TypOgloszeniodawcy = TypOgloszeniodawcy.text.strip() if TypOgloszeniodawcy else "N/A"

        DostepneOd = body.find('div', {'aria-label': 'Dostępne od', 'class': 'css-tpkder e26jmad1'})
        DostepneOd = DostepneOd.text.strip() if DostepneOd else "N/A"

        RokBudowy = body.find('div', {'aria-label': 'Rok budowy', 'class': 'css-tpkder e26jmad1'})
        RokBudowy = RokBudowy.text.strip() if RokBudowy else "N/A"

        RodzajZabudowy = body.find('div', {'aria-label': 'Rodzaj zabudowy', 'class': 'css-tpkder e26jmad1'})
        RodzajZabudowy = RodzajZabudowy.text.strip() if RodzajZabudowy else "N/A"
        
        Okna = body.find('div', {'aria-label': 'Okna', 'class': 'css-tpkder e26jmad1'})
        Okna = Okna.text.strip() if Okna else "N/A"

        CzyWinda = body.find('div', {'aria-label': 'Winda', 'class': 'css-tpkder e26jmad1'})
        CzyWinda = CzyWinda.text.strip() if CzyWinda else "N/A"
        
        Media = body.find('div', {'aria-label': 'Media', 'class': 'css-tpkder e26jmad1'})
        Media = Media.text.strip() if Media else "N/A"

        Zabezpieczenia = body.find('div', {'aria-label': 'Zabezpieczenia', 'class': 'css-tpkder e26jmad1'})
        Zabezpieczenia = Zabezpieczenia.text.strip() if Zabezpieczenia else "N/A"

        Wyposazenie = body.find('div', {'aria-label': 'Wyposażenie', 'class': 'css-tpkder e26jmad1'})
        Wyposazenie = Wyposazenie.text.strip() if Wyposazenie else "N/A"

        InformacjeDodatkowe = body.find('div', {'aria-label': 'Informacje dodatkowe', 'class': 'css-tpkder e26jmad1'})
        InformacjeDodatkowe = InformacjeDodatkowe.text.strip() if InformacjeDodatkowe else "N/A"

        MaterialBudynku = body.find('div', {'aria-label': 'Materiał budynku', 'class': 'css-tpkder e26jmad1'})
        MaterialBudynku = MaterialBudynku.text.strip() if MaterialBudynku else "N/A"

        rooms = {
            'ID': NumerId,
            'Nazwa': nazwa,
            'CenaM2': cenaM2,
            'Cena': cena,
            'Adres': adres,
            'Powierzchnia': powierzchnia,
            'Forma_Wlasnosci': formaWlasnosci,
            'LiczbaPokoi': LiczbaPokoi,
            'StanWykonczenia': stanWykonczenia,
            'Pietro': pietro,
            'BalkonOgrodTaras': BalkonOgrodTaras,
            'Czynsz': Czynsz,
            'Ogrzewanie': ogrzewanie,
            'Rynek': Rynek,
            'TypOgloszeniodawcy': TypOgloszeniodawcy,
            'DostepneOd': DostepneOd,
            'RokBudowy': RokBudowy,
            'RodzajZabudowy': RodzajZabudowy,
            'CzyWinda': CzyWinda,
            'Zabezpieczenia': Zabezpieczenia,
            'Wyposazenie': Wyposazenie,
            'InformacjeDodatkowe': InformacjeDodatkowe,
            'MaterialBudynku': MaterialBudynku
        }
        houselist.append(rooms)
    except AttributeError as e:
        #print(f"Error parsing details for {link}: {e}")
        continue

# Save the data to a DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(houselist)
df

Page 4236 status code: 200
Found 39 listing containers on page 4236
Page 4237 status code: 200
Found 39 listing containers on page 4237
Page 4238 status code: 200
Found 39 listing containers on page 4238
Page 4239 status code: 200
Found 39 listing containers on page 4239
Page 4240 status code: 200
Found 6 listing containers on page 4240
Page 4241 status code: 200
No advertisements found. Stopping the scraping process.
Total houselinks found: 162


,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,Rynek,TypOgloszeniodawcy,DostepneOd,RokBudowy,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku
0,https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-w-bloku-48m2-z-piwnica-ID4r5Aa,2 pokojowe mieszkanie w bloku 48m2 z piwnicą,7 688 zł/m²,369 000 zł,"ul. Złota, Piskorzewie, Kalisz, wielkopolskie",Powierzchnia48 m²,Forma własnościpełna własność,Liczba pokoi2,Stan wykończeniado zamieszkania,Piętro1/7,...,Rynekwtórny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowybrak informacji,Rodzaj zabudowyblok,Windatak,"Zabezpieczeniasystem alarmowy, drzwi / okna antywłamaniowe, teren zamknięty, domofon / wideofon, monitoring / ochrona, rolety antywłamaniowe","Wyposażeniezmywarka, lodówka, meble, piekarnik, kuchenka, telewizor, pralka","Informacje dodatkowepiwnica, oddzielna kuchnia, pom. użytkowe",Materiał budynkubrak informacji
1,https://www.otodom.pl/pl/oferta/sprzedam-wynajme-mieszkanie-ID4qVDh,sprzedam/wynajmę mieszkanie,6 984 zł/m²,440 000 zł,"ul. Wincentego Witosa, Chrzanów, Chrzanów, chrzanowski, małopolskie",Powierzchnia63 m²,Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętro4/4,...,Rynekwtórny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowy1980,Rodzaj zabudowyblok,Windanie,"Zabezpieczeniadrzwi / okna antywłamaniowe, domofon / wideofon","Wyposażeniezmywarka, lodówka, meble, piekarnik, kuchenka, pralka","Informacje dodatkoweklimatyzacja, piwnica, oddzielna kuchnia",Materiał budynkuwielka płyta
2,https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-45m2-loggia-ID4hMds,2-pokojowe mieszkanie 45m2 + loggia,12 533 zł/m²,567 000 zł,"ul. Wagrowska, Starołęka Mała, Nowe Miasto, Poznań, wielkopolskie","Powierzchnia45,24 m²",Forma własnościpełna własność,Liczba pokoi2,Stan wykończeniado wykończenia,Piętro> 10/17,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,https://www.otodom.pl/pl/oferta/al-sw-wojciecha-42-m2-2-pok-oddane-do-uzytku-ID47Ylm,Al. Św. Wojciecha 42 m2. 2 pok. Oddane do użytku!,6 700 zł/m²,279 524 zł,"Łomia, Lipowiec Kościelny, mławski, mazowieckie","Powierzchnia41,72 m²",Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado wykończenia,Piętro3/4,...,Rynekpierwotny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne odbrak informacji,Rok budowy2020,Rodzaj zabudowyblok,Windatak,"Zabezpieczeniadrzwi / okna antywłamaniowe, teren zamknięty, domofon / wideofon",Wyposażeniebrak informacji,Informacje dodatkowebrak informacji,Materiał budynkusilikat
4,https://www.otodom.pl/pl/oferta/mieszkanie-w-przedwojennym-stylu-lub-na-kancelarie-ID47NjQ,Mieszkanie w przedwojennym stylu lub na kancelarię,3 537 zł/m²,520 000 zł,"Śródmieście, Bydgoszcz, kujawsko-pomorskie",Powierzchnia147 m²,Forma własnościpełna własność,Liczba pokoi4,Stan wykończeniaZapytaj,Piętro1/3,...,Rynekwtórny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne odbrak informacji,Rok budowybrak informacji,Rodzaj zabudowykamienica,Windanie,Zabezpieczeniabrak informacji,Wyposażeniebrak informacji,"Informacje dodatkowepiwnica, oddzielna kuchnia",Materiał budynkucegła
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,https://www.otodom.pl/pl/oferta/atrakcyjne-mieszkanie-2-pokojowe-rezerwacja-ID4pZRb,Atrakcyjne mieszkanie 2-pokojowe REZERWACJA,7 000 zł/m²,343 000 zł,"ul. Henryka Sienkiewicza, Śródmieście, Radom, mazowieckie",Powierzchnia49 m²,Forma własnościpełna własność,Liczba pokoi2,Stan wykończeniado zamieszkania,Piętro1/3,...,Rynekwtórny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowy1960,Rodzaj zabudowyblok,Windanie,"Zabezpieczeniateren zamknięty, domofon / wideofon","Wyposażeniezmywarka, lodówka, meble, piekarnik, kuchenka, telewizor, pralka","Informacje dodatkowepiwnica, oddzielna kuchnia",Materiał budynkucegła
158,https://www.otodom.pl/pl/oferta/1-pokojowe-mieszkanie-33m2-loggia-ID4pn9L,1-pokojowe mieszkanie 33m2 + logg

In [324]:
df_copy = df.copy()

In [221]:
## FEATURE ENGINEERING

In [261]:
## ID
df_copy['ID'] = df_copy['ID'].apply(lambda x: re.search(r'(ID\w+)', x).group(0) if re.search(r'(ID\w+)', x) else None)

In [262]:
# Cena
df_copy['Cena'] = df_copy['Cena'].apply(lambda y: str(y).split('zł')[0])
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace(' ', ''))
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace(',', '.'))
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace('Zapytajocenę',''))
df_copy['Cena'] = df_copy['Cena'].replace('', np.nan)
df_copy['Cena'] = df_copy['Cena'].astype(float)


In [263]:
# CenaM2
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda y: str(y).split('zł/m²')[0])
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda x: x.replace(' ', ''))
df_copy['CenaM2'] = df_copy['CenaM2'].replace('', np.nan)
df_copy['CenaM2'] = df_copy['CenaM2'].astype(float)

In [225]:
## Nazwa czy potrzebna ?

In [264]:
## Adres
### Wyodrębnienie nazwy wojewodztwa

wojewodztwa = ['dolnośląskie','kujawsko-pomorskie','lubelskie','lubuskie','łódzkie',
               'małopolskie','mazowieckie','opolskie','podkarpackie','podlaskie','pomorskie',
               'śląskie','świętokrzyskie','warmińsko-mazurskie','wielkopolskie','zachodniopomorskie']



In [265]:
df_copy['Wojewodztwo'] = df_copy['Adres'].str.split().apply(lambda x: '_'.join([m for m in x if m in wojewodztwa])).replace('',np.nan)

In [266]:
### Wyodrębnienie nazwy miast
file_name = r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListOfCity\SIMC_15-06-2024 ({}).csv'
df_city_poland = pd.concat([pd.read_csv(file_name.format(i),sep=';') for i in range(0, 16)])

In [267]:
city_set = {city.strip().lower() for city in df_city_poland['NAZWA']}

def extract_city_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    city = next((part for part in parts if part in city_set), np.nan)
    return city.capitalize() if city is not np.nan else city

df_copy['Miasto'] = df_copy['Adres'].apply(extract_city_values)


In [230]:
### Wyodrębnienie nazwy powiatow

In [268]:
df_district_poland = pd.read_excel(r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListofDistrict\ListaPowiatow.xlsx')

In [269]:
district_set = {district.strip().lower() for district in df_district_poland['Powiat']}

def extract_district_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    district = next((part for part in parts if part in district_set), np.nan)
    return district.capitalize() if district is not np.nan else district

df_copy['Powiat'] = df_copy['Adres'].apply(extract_district_values)

In [270]:
df_copy[['Miasto','Adres','Powiat','Wojewodztwo']]

,Miasto,Adres,Powiat,Wojewodztwo
0,Kalisz,"ul. Złota, Piskorzewie, Kalisz, wielkopolskie",Kalisz,wielkopolskie
1,Chrzanów,"ul. Wincentego Witosa, Chrzanów, Chrzanów, chrzanowski, małopolskie",Chrzanowski,małopolskie
2,Nowe miasto,"ul. Wagrowska, Starołęka Mała, Nowe Miasto, Poznań, wielkopolskie",Poznań,wielkopolskie
3,NaN,"Łomia, Lipowiec Kościelny, mławski, mazowieckie",Mławski,mazowieckie
4,Bydgoszcz,"Śródmieście, Bydgoszcz, kujawsko-pomorskie",Bydgoszcz,kujawsko-pomorskie
...,...,...,...,...
157,Radom,"ul. Henryka Sienkiewicza, Śródmieście, Radom, mazowieckie",Radom,mazowieckie
158,Warszawa,"ul. Anny German, Sady Żoliborskie, Żoliborz, Warszawa, mazowieckie",Warszawa,mazowieckie
159,Inowrocław,"ul. ks. Piotra Wawrzyniaka, Inowrocław, inowrocławski, kujawsko-pomorskie",Inowrocławski,kujawsko-pomorskie
160,Sopot,"ul. Fryderyka Chopina, Dolny Sopot - Grunwaldzka, Dolny Sopot, Sopot, pomorskie",Sopot,pomorskie


In [234]:
### Wyodrębnienie nazwy ulicy

In [271]:
def extract_ulica_values(address):
    matches = re.findall(r'\bul\.\s*[^,]*', address)
    return ', '.join(matches) if matches else np.nan

df_copy['Ulica'] = df_copy['Adres'].apply(extract_ulica_values)

In [272]:
##Powierzchnia

df_copy['Powierzchnia'] = df_copy['Powierzchnia'].str.replace('Powierzchnia', '').str.strip()
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].apply(lambda y: str(y).split('m²')[0])
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].apply(lambda x: x.replace(',', '.'))
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].astype(float)

In [273]:
##Forma_Wlasnosci

df_copy['Forma_Wlasnosci'] = df_copy['Forma_Wlasnosci'].str.replace('Forma własności', '').str.strip()

In [274]:
##LiczbaPokoi
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].str.replace('Liczba pokoi', '').str.strip()
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].astype(int)

In [275]:
##StanWykonczenia
df_copy['StanWykonczenia'] = df_copy['StanWykonczenia'].str.replace('Stan wykończenia', '').str.strip()

In [325]:
##Pietro
df_copy['Pietro'] = df_copy['Pietro'].str.replace('Piętro', '').str.strip()
df_copy['Pietro'] = df_copy['Pietro'].str.replace('parter', '0')
df_copy['Pietro'] = df_copy['Pietro'].replace('poddasze', '100')
df_copy['Pietro'] = df_copy['Pietro'].replace('Zapytaj', '')
df_copy['Pietro'] = df_copy['Pietro'].replace('>', '')
df_copy['Pietro'] = df_copy['Pietro'].replace('', np.nan)
df_copy['Pietro'] = df_copy['Pietro'].apply(lambda x: int(x.split('/')[0]) if isinstance(x, str) and '/' in x else x)


ValueError: invalid literal for int() with base 10: '> 10'

In [241]:
##RodzajZabudowy
df_copy['RodzajZabudowy'] = df_copy['RodzajZabudowy'].str.replace('Rodzaj zabudowy', '').str.strip()

In [242]:
##CzyWinda
df_copy['CzyWinda'] = df_copy['CzyWinda'].str.replace('Winda', '').str.strip()

In [243]:
##Zabezpieczenia
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].str.replace('Zabezpieczenia', '').str.strip()

In [244]:
##Wyposazenie
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].str.replace('Wyposażenie', '').str.strip()

In [245]:
##InformacjeDodatkowe
df_copy['InformacjeDodatkowe'] = df_copy['InformacjeDodatkowe'].str.replace('Informacje dodatkowe', '').str.strip()

In [246]:
##MaterialBudynku
df_copy['MaterialBudynku'] = df_copy['MaterialBudynku'].str.replace('Materiał budynku', '').str.strip()

In [322]:
df_copy['Pietro'].value_counts()

0.0      6
1.0      4
3.0      2
2.0      1
100.0    1
10.0     1
4.0      1
Name: Pietro, dtype: int64